In [48]:
%matplotlib inline
import numpy as np

import os
import sys
BASE_PATH = os.path.dirname(os.getcwd())
sys.path.append(BASE_PATH)
import matplotlib.pyplot as plt

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import backend as K

from korean_ocr.layers import *
from korean_ocr.data.generator import DataGenerator
from korean_ocr.data.dataset import read_label_dataframe
from korean_ocr.utils.serving import convert_model_to_inference_model
from korean_ocr.utils.jamo import compose_unicode

# \[ 한글 데이터셋 구성하기 \]
---
----

## 1. 한글 matplotlib 출력 세팅
---

Jupyter Notebook과 Matplotlib은 기본적으로 한글을 지원하지 않습니다. 한글이 출력되려면 아래와 같이 세팅을 해주어야 합니다.

````bash
# 나눔 폰트 설치하기
!apt-get update -qq
!apt-get install fonts-nanum* -qq
````

In [ ]:
import matplotlib.font_manager as fm

# 1. 나눔 폰트의 위치 가져오기 
system_font = fm.findSystemFonts() # 현재 시스템에 설치된 폰트
nanum_fonts = [
    font for font in system_font if "NanumBarunGothic.ttf" in font]
font_path = nanum_fonts[0] # 설정할 폰트의 경로

# 2. 나눔 폰트로 설정하기
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc("font",family=font_name)

# 3. 폰트 재설정하기
fm._rebuild()

# 4. (optional) minus 기호 깨짐 방지
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

## 2. Dataset 구성하기
----

한국정보화진흥원(NIA)에서 제공하고 있는 데이터 셋은 크게 두가지가 있습니다.

1. 손글씨 데이터 셋 : 
2. 인쇄 데이터 셋 :

In [ ]:
from korean_ocr.data.dataset import read_label_dataframe, filter_out_dataframe
from sklearn.model_selection import train_test_split

dataset_dir = "../datasets/handwritten"
label_path = os.path.join(dataset_dir, "dataset_info.json")

label_df = read_label_dataframe(label_path)
label_df = filter_out_dataframe(label_df)

# Train Dataset과 Validation Dataset을 구분
train_df, valid_df = train_test_split(
    label_df,test_size=0.1, random_state=42)

train_df.head(10)

위의 정보에 따라, 이미지와 라벨을 같이 읽어오는 클래스로 `OCRDataset`이 있습니다. 아래와 같이 이용할 수 있습니다.

In [ ]:
from korean_ocr.data.dataset import OCRDataset

# OCR DATASET 구성하기
height = 64

trainset = OCRDataset(train_df, height)
validset = OCRDataset(valid_df, height)

print("train data의 갯수 : ",len(trainset))
print("valid data의 갯수 : ",len(validset))

In [ ]:
# 복수개의 이미지 가져오기
images, texts = trainset[4:7]

for image, text in zip(images, texts):
    plt.title(text)
    plt.imshow(image,cmap='gray')
    plt.xticks([]); plt.yticks([])
    plt.show()

## 3. Data Generator 만들기
----

위의 데이터 셋은 모델이 학습할 수 있는 형태로 가공 후, 배치 단위로 모델에게 주입시켜야 합니다. 이러한 작업은 아래의 `DataGenerator`에서 담당합니다.

In [ ]:
from korean_ocr.data.generator import DataGenerator
from korean_ocr.utils.jamo import compose_unicode

traingen = DataGenerator(trainset, batch_size=32)

Generator에서는 아래와 같이, Text를 Unicode 숫자로 변환 후 반환합니다. 그리고 배치 단위 학습을 위해, 이미지의 크기가 배치 단위로 모두 같도록 가장 가로로 긴 이미지를 기준으로, 가로 방향으로 검은색(pixel=0) 패딩을 채워줍니다.

In [ ]:
sample_inputs, sample_outputs = traingen[4]
sample_images = sample_inputs['images']
sample_decoder_inputs = sample_inputs['decoder_inputs']

for i in range(5):
    image = sample_images[i]
    plt.title('입력 이미지')
    plt.imshow(image,cmap='gray')
    plt.xticks([]); plt.yticks([])    
    plt.show()    
    print("outputs(text)    : ", compose_unicode(sample_outputs[i])[0])
    print("outputs(unicode) : ", sample_outputs[i])

# \[ 모델 구성하기 \]
---
----



## 1. 전처리 파이프라인 추가
----

In [ ]:
from korean_ocr.layers import PreprocessImage
from tensorflow.keras.layers import Input

K.clear_session()
height = traingen.dataset.height

inputs = Input(shape=(None, None), name='images')
preprocess_layer = PreprocessImage(height=height, normalize=True,
                                   name='encoder/preprocess')

prep_out, prep_masks = preprocess_layer(inputs)

In [ ]:
# Test하기 위한 코드
run = lambda x: K.get_session().run(x, {inputs:sample_images})

#### 출력값의 크기는 아래와 같습니다.

In [ ]:
print("Inputs Shape : ", inputs.get_shape())
print("prep_out shape : ", prep_out.get_shape())
print("prep_masks shape : ", prep_masks.get_shape())

In [ ]:
fig = plt.figure(figsize=(8,2))
n = 5
for i in range(1,n+1):
    ax = fig.add_subplot(2,n,i)
    ax.imshow(run(prep_out)[i,...,0],cmap='gray')
    ax.set_xticks([]); ax.set_yticks([])
    ax2 = fig.add_subplot(2,n,i+n)
    ax2.imshow((run(prep_masks)[i,...,0]),cmap='gray')
    ax2.set_xticks([]); ax2.set_yticks([])
plt.show()

## 2. Convolution Feature Extractor 추가하기

**출처 : Baoguang Shi, Robust Scene Text Recognition with Automatic Rectification**

> In the SRN, the encoder has 7 convolutional layers, whose {filter size, number of fileters, stride, padding size} are respectively {3, 64, 1, 1}, {3, 128, 1, 1}, {3, 256, 1, 1}, {3, 256, 1, 1}, {3, 512, 1, 1}, {3, 512, 1, 1} and {2, 512, 1, 0}. The 1st, 2nd, 4th, 6th convolutional layers are each followed by a 2 x 2 max-pooling layer.

우리는 위에서 제시한 모델에서 보다 빠른 수렴을 위해, Residual Connection과 Batch Normalization을 추가하였습니다. 그리고 한글은 조합형 언어이기 때문에, 종방향의 해상도가 좀 더 필요하다고 판단하여, height을 32가 아닌 64으로 설정하였습니다.

In [ ]:
conv_layer = ConvFeatureExtractor(name='encoder/feature_extractor')
conv_maps, conv_masks = conv_layer([prep_out, prep_masks])

#### 출력값의 크기는 아래와 같습니다.

In [ ]:
print("Inputs의 Shape : ", inputs.get_shape())
print("-> conv maps의 Shape : ", conv_maps.get_shape())
print("-> conv masks의 Shape : ", conv_masks.get_shape())

print("\n\n실제 Sample Input의 Shape : ",sample_inputs['images'].shape)
print("-> Conv Map의 Shape : ",
      Model(inputs, conv_maps).predict(sample_inputs['images']).shape)
print("-> Conv Masks의 Shape : ",run(conv_masks).shape)

In [ ]:
ex_conv_maps = run(conv_maps)
fig = plt.figure(figsize=(8,2))
n = 5
for i in range(1,n+1):
    ax = fig.add_subplot(2,n,i)
    ax.imshow(ex_conv_maps[i,...,:3]/ex_conv_maps[i,...,:3].max(), cmap='gray')
    ax.set_xticks([]); ax.set_yticks([])
    ax2 = fig.add_subplot(2,n,i+n)
    ax2.imshow((run(conv_masks)[i,...,0]),cmap='gray')
    ax2.set_xticks([]); ax2.set_yticks([])
plt.show()

## 2. Map2Sequence 구성하기
----

**출처 : Baoguang Shi, Robust Scene Text Recognition with Automatic Rectification**

> Specifically, the "map-to-sequence" operation takes out the columns of the maps in the left-to-right order, and flattens them into vectors. According to the translation invariance property of CNN, each vector corresponds to a local image region, i.e. receptiv field, and is a descriptor for that region.

이미지 데이터는 4차원(batch, height, width, channel)으로 이루어져 있고, 텍스트 정보는 3차원(batch, width, height * channel)으로 이루어져 있습니다. 우리는 이미지 데이터를 통해 텍스트 정보를 추출해야 하므로, 4차원 정보를 3차원 정보로 변환해야 합니다. 이러한 작업을 수행하기 위해, Map2Sequence 연산을 구현하여 적용하였습니다.


In [ ]:
from korean_ocr.layers import Map2Sequence

m2s_layer = Map2Sequence(name='map_to_sequence')
feat_maps, feat_masks = m2s_layer([conv_maps,conv_masks])

#### 출력값의 크기는 아래와 같습니다.

In [ ]:
print("conv maps의 Shape : ", conv_maps.get_shape())
print("-> feature maps의 Shape : ", feat_maps.get_shape())

print("\n\n실제 Conv Map의 Shape : ",
      Model(inputs, conv_maps).predict(sample_inputs).shape)
print("-> feature maps의 Shape : ",
      Model(inputs, feat_maps).predict(sample_inputs).shape)
print("-> feat masks의 Shape : ",
      run(feat_masks).shape)

## 3. Sequence Encoder 구성하기
----

**출처 : Baoguang Shi, Robust Scene Text Recognition with Automatic Rectification**

> Restricted by the sizes of the receptive fields, the feature sequence leverages limited image contexts. We further apply a two-layer Bidrectional Long-Short Term Memory(BLSTM) Network to the sequence, in order to model the long-term dependencies within the sequence. The BLSTM is a recurrent network that can analyze the dependencies within a sequence in both directions, it outputs another sequence which has the same length as the input one. The output sequence is $h=(h_1, ...,h_L)$, where $L=W_{conv}$. 

CNN Network는 Receptive Field에 한정되어서만 정보를 읽어들이고, 이미지의 순서를 해석하지 못하기 때문에, 위의 문제들을 해결하기 위해 Sequence을 읽어들일 수 있는 BLSTM Layer을 추가하였습니다. 


In [ ]:
from korean_ocr.layers import SequenceEncoder

num_depth = 2
num_states = 128

rnn_seqs = SequenceEncoder(
    recurrent_cell='lstm', num_depth=num_depth,
    num_states=num_states)([feat_maps,feat_masks])

#### 출력값의 크기는 아래와 같습니다.

In [ ]:
print("feature maps의 Shape : ", feat_maps.get_shape())
print("->rnn sequences의 Shape : ", rnn_seqs.get_shape())

print("\n\n실제 feature maps의 Shape : ",
      Model(inputs, feat_maps).predict(sample_inputs).shape)
print("-> rnn sequences의 Shape : ",
      Model(inputs, rnn_seqs).predict(sample_inputs).shape)

## 4. Decoder Input Embedding 구성하기
----

초성 / 중성 / 종성 / 특수문자로 나누어 임베딩하는 방식으로 진행하였습니다.

In [ ]:
from korean_ocr.layers.text import CharEmbedding

decoder_inputs = Input(shape=(None,), dtype=tf.int32, 
                       name='decoder_inputs')

embedding_layer = CharEmbedding()
dec_embeded, dec_masks = embedding_layer(decoder_inputs)

In [ ]:
# Test하기 위한 코드
run = lambda x: K.get_session().run(x, {inputs:sample_images,decoder_inputs:sample_decoder_inputs})

#### 출력값의 크기는 아래와 같습니다.

In [ ]:
print("decoder inputs의 Shape : ", decoder_inputs.get_shape())
print("dec_embeded의 Shape : ", dec_embeded.get_shape())
print("dec_masks의 Shape : ", dec_masks.get_shape())

print("\n\n실제 decoder inputs의 Shape : ", sample_decoder_inputs.shape)
print("dec_embeded의 Shape : ", run(dec_embeded).shape)
print("dec_masks의 Shape : ", run(dec_masks).shape)

## 5. Attention Decoder 구성하기
----

**출처 : Baoguang Shi, Robust Scene Text Recognition with Automatic Rectification**

> The generation is a T-step process, at step $t$, the decoder computes a vector of attention weights $\alpha_t \in R^L$ via the attention process described in $\alpha_t = Attend(s_{t-1}, \alpha_{t-1}, h)$  where $s_{t-1}$ is the state variable of the GRU cell at the last step. For $t=1$, both $s_0$ and $\alpha_0$ are zero vectors. Then, a glimpse $g_t$ is computed by linearly combining the vectors in h: $g_t = \sum^L_{i=1} \alpha_{ti}h_i$. Since $\alpha_t$ has non-negative values that sum to one, it effectively controls where the decoder focused on. The state $s_{t-1}$ is updated via teh recurrent process of GRU: $s_t=GRU(l_{t-1}, g_t. s_{t-1})$. where $l_{t-1}$ is the (t-1) th ground-truth label in training, while in testing, it is the label predicted in the previous step, i.e. $\hat l_{t-1}$. The probability distribution over the label space is estimated by: $\hat y_t = softmax(W^t s_t).$ Following that, a character $\hat l_t$ is predicted by taking the class with the highest probability. The label space includes all English alphanumeric characters, plus a special "end-of-sequence"(EOS) token, which ends the generation process.


1. GRU Network : 글자 영상에서 순서에 맞게 텍스트를 해석
2. Attention Network : GRU Network에서 필요한 정보만을 추출

위 두 Network의 조합으로 Character Decoder를 구성하였습니다.

In [ ]:
from korean_ocr.layers import AttentionDecoder
attend_decoder = AttentionDecoder(num_states=num_states)

states = attend_decoder([rnn_seqs, dec_embeded, feat_masks, dec_masks])

#### 출력값의 크기는 아래와 같습니다.

In [ ]:
print("rnn sequences의 Shape : ", rnn_seqs.get_shape())
print("->attention state의 Shape : ", states.get_shape())

sample_label = np.array([[1,3,5,2,1]])

print("\n\n실제 rnn sequences의 Shape : ",
      Model(inputs,
            rnn_seqs).predict(sample_inputs).shape)
print("-> attention state의 Shape : ",
      Model([inputs, decoder_inputs],
            states).predict(sample_inputs).shape)

## 6. Chararcter Classification Layer 구성하기
---

한글 완성형 글자 수 (11,172자)와 EOS(End-Of-Sequence) 토큰을 더해 총 11,173자의 글자를 분류해야 합니다. 이경우 매우 Sparse하기 때문에, 모델의 학습에 문제가 발생할 수 있습니다. 이를 방지하기 위해서 우리는 한글이 조합형 글자라는 특징을 살려, 초성 / 중성 / 종성을 각각 나누어 Classification하고, 이를 합치는 방식으로 재구성하였습니다.

In [ ]:
from korean_ocr.layers import CharClassifier

char_classifer = CharClassifier(num_fc=128)
prediction = char_classifer(states)

#### 출력값의 크기는 아래와 같습니다.

In [ ]:
print("attention state의 Shape : ", states.get_shape())
print("->output의 Shape : ", prediction.get_shape())

sample_label = np.array([[1,3,5,2,1]])

print("\n\n실제 attention state의 Shape : ",
      Model([inputs, decoder_inputs], states).predict(sample_inputs).shape)
print("-> output의 Shape : ",
      Model([inputs, decoder_inputs], prediction).predict(sample_inputs).shape)

# \[ 모델 학습하기 \]
---
----


## 1. 모델 구성하기
---

위에서 설계한 모델은 아래의 코드로 생성할 수 있습니다.

In [ ]:
from korean_ocr.model import build_ocr_model

K.clear_session()
ocr_model = build_ocr_model()
ocr_model.summary()

## 2. 모델 컴파일하기
----

모델에 학습에 필요한 옵티마이저, 목적 함수, 그리고 평가지표 등을 설정합니다.

In [ ]:
from korean_ocr.model import compile_ocr_model

ocr_model = compile_ocr_model(ocr_model)

## 3. 모델 학습시키기
---


In [ ]:
from korean_ocr.model import train_ocr_model

train_ocr_model(ocr_model, data_dir=["../datasets/handwritten/",
                                     "../datasets/printed/"])